In [0]:
import torch
from torch import Tensor, LongTensor, FloatTensor
import math
import modules as m

In [0]:
def generate_disc_data(n = 1000):
  '''
  Generates disc data, sampled uniformly in [0,1]², around center C(1/2,1/2) and radius r = 1/sqrt(2*pi)
  '''

  center = torch.Tensor([[0.5, 0.5]])
  input_coords = torch.rand(n,2)
  L2_distances = torch.norm((input_coords - center).abs(), 2, 1, True)
  targets = L2_distances.mul(math.sqrt(2*math.pi)).sub(1).sign().sub(1).div(2).abs().long()
  return input_coords, targets

In [0]:
def split_data(inputs, targets, train_ratio = 0.7, validation_ratio = 0.1, test_ratio = 0.2):
  '''
  Splits the data into a training, a validation and a test set
  '''

  train_size = math.floor(inputs.size()[0] * train_ratio)
  validation_size = math.floor(inputs.size()[0] * validation_ratio)
  test_size = math.floor(inputs.size()[0] * test_ratio)

  train_inputs  = inputs.narrow(0, 0, train_size)
  train_targets = targets.narrow(0, 0, train_size)

  validation_inputs  = inputs.narrow(0, train_size, validation_size)
  validation_targets = targets.narrow(0, train_size, validation_size)

  test_inputs  = inputs.narrow(0, train_size+validation_size, test_size)
  test_targets = targets.narrow(0, train_size+validation_size, test_size)

  return train_inputs, train_targets, validation_inputs, validation_targets, test_inputs, test_targets


In [0]:
def convert_to_one_hot_labels(input, target):
    '''
    Converts targets to one-hot labels of values -1 and 1
    '''
    
    tmp = input.new(target.size(0), target.max() + 1).fill_(-1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [0]:
def train_model(train_input, train_target, validation_input, validation_target, model, optimizer, nb_epochs = 100):
  '''
  Trains the model with the given optimizer, outputs the trained model along with the train and validation error lists,
  prints both train and validation errors
  '''

  nb_train_samples = train_input.size(0)
  nb_val_samples = validation_input.size(0)

  #Initialization of the output error lists
  train_error      = []
  validation_error = []

  #Convert target values using one hot encoding
  train_target = convert_to_one_hot_labels(train_input, train_target)
  validation_target = convert_to_one_hot_labels(validation_input, validation_target)


  for e in range(nb_epochs):

    #Training

    loss = 0
    nb_train_errors = 0

    for n in range(nb_train_samples):
      
      optimizer.zero_grad()
      
      target = train_target[n]
      target_tuple = [target[0], target[1]]
      target_index = target_tuple.index(max(target_tuple))

      pred = model.forward(train_input[n])
      pred_tuple = [pred[0], pred[1]]
      pred_index = pred_tuple.index(max(pred_tuple))

      if int(target_index) != int(pred_index) : nb_train_errors += 1
      
      loss += m.MSELoss(target.float(), pred) 
      dl_loss = m.dl_MSELoss(target.float(), pred)

      model.backward(dl_loss)

      optimizer.step()

    train_error.append((100 * nb_train_errors) / nb_train_samples)


    #Validation

    nb_validation_errors = 0

    for n in range(nb_val_samples):

      target = validation_target[n]
      target_tuple = [target[0], target[1]]
      target_index = target_tuple.index(max(target_tuple))

      pred = model.forward(validation_input[n])
      pred_tuple = [pred[0], pred[1]]
      pred_index = pred_tuple.index(max(pred_tuple))

      if int(target_index) != int(pred_index) : nb_validation_errors += 1
    
    validation_error.append((100 * nb_validation_errors) / nb_val_samples)
  

    #Print intermediate loss and error results (each 10 epochs)

    if (e % 10) == 0:
      print('Epoch : {:d}, Train_loss {:.02f}, Train_error {:.02f}%, Validation_error {:.02f}%'
      .format(e, loss, (100 * nb_train_errors)/nb_train_samples, (100 * nb_validation_errors)/nb_val_samples))

  return model, train_error, validation_error

In [0]:
def test_model(trained_model, test_input, test_target):
  '''
  Tests the trained model with the new data, prints test error
  '''

  nb_test_samples = test_input.size(0)
  test_target = convert_to_one_hot_labels(test_input, test_target)
  nb_test_errors = 0

  for n in range(nb_test_samples):

    target = test_target[n]
    target_tuple = [target[0], target[1]]
    target_index = target_tuple.index(max(target_tuple))

    pred = trained_model.forward(test_input[n])
    pred_tuple = [pred[0], pred[1]]
    pred_index = pred_tuple.index(max(pred_tuple))

    if int(target_index) != int(pred_index) : nb_test_errors += 1

  print('test_error {:.02f}%'.format(((100 * nb_test_errors) / nb_test_samples)))

  return